In [7]:
''' 
Aufgaben die zu erledigen sind:
- Product Shelf bereinigen --> schlecht gekaufte Produkte
- Produkte empfehlen die noch nicht eingekauft wurden

3 mögliche Systeme:
- Simple Recommender --> Top Products
    - Decide on the metric or score to rate products on
    - Calcualte the score for every product
    - Sort the products based on the score and output the top results
    
- Content Based Recommender --> Uses item meta data
- Collaborative Recommender --> Products from other user that looks pretty similar


Problems:
Contrarian to rating data, purchase data has no upper band of the evaluated score
There is no explicit rating data, so it has to be generated from implicit information available

Ressources:
- https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
- https://www.datacamp.com/community/tutorials/recommender-systems-python?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=229765585186&utm_targetid=aud-299261629574:dsa-473406569915&utm_loc_interest_ms=&utm_loc_physical_ms=20143&gclid=Cj0KCQiA-bjyBRCcARIsAFboWg2VIh0dOVFJAckISmcG52iT4y1gr732tQkw-i9-Pj2VubR5WXq4IWcaArhsEALw_wcB


''' 

' \nAufgaben die zu erledigen sind:\n- Product Shelf bereinigen --> schlecht gekaufte Produkte\n- Produkte empfehlen die noch nicht eingekauft wurden\n\n3 mögliche Systeme:\n- Simple Recommender --> Top Products\n    - Decide on the metric or score to rate products on\n    - Calcualte the score for every product\n    - Sort the products based on the score and output the top results\n    \n- Content Based Recommender --> Uses item meta data\n- Collaborative Recommender --> Products from other user that looks pretty similar\n\n\nProblems:\nContrarian to rating data, purchase data has no upper band of the evaluated score\n\nRessources:\n- https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6\n- https://www.datacamp.com/community/tutorials/recommender-systems-python?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=2

In [1]:
import numpy as np
import pandas as pd


raw_data = pd.read_csv ("Recommender4Retail.csv") # low_memory option is depricated
raw_data.head()


,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,soft drinks
1,2,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,dairy eggs,soy lactosefree
2,3,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,snacks,popcorn jerky
3,4,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,snacks,popcorn jerky
4,5,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,household,paper goods


In [3]:
# Some EDA

#raw_data.info()
#raw_data.describe()

'''
- What is eval_set, order_dow, reordered and can they be used to create a rating?
- Can the amount a product has been bought, seen as a implicit ratings? There might be some problems to it depending on the product...
'''

#Unique Users
#unique_users = raw_data.drop_duplicates(subset=['user_id'], keep='first')

#unique_users = raw_data['user_id'].unique()

#unique_users.count()

# filter a user
user1 = raw_data[(raw_data.user_id == 1)] # data_limited[(data_limited.user_id == 1) & (data_limited.product_id == 196)]

# amount of products bought by user
n_by_user = raw_data.groupby("user_id")["product_id"].count()
print(n_by_user)

# number of times a product is bought
n_of_products = raw_data.groupby("product_id")["user_id"].count()
print(n_of_products)


user_id
1          70
2         226
3          88
4          18
5          46
         ... 
206205     51
206206    285
206207    223
206208    677
206209    137
Name: product_id, Length: 206209, dtype: int64
product_id
1        1928
2          94
3         283
4         351
5          16
         ... 
49684       9
49685      49
49686     127
49687      14
49688      93
Name: user_id, Length: 49685, dtype: int64


In [5]:
# create a matrix with user, product and occurence as value (= number of times purchased)

data_limited = raw_data.head(5000) # reduced for testing purposes
matrix = pd.crosstab(data_limited.user_id, raw_data.product_id) 
# if we want precentage add parameter normalize=True

# this could alternatively be done by groupby (or pivot_table)
matrix_g = data_limited.groupby(['user_id', 'product_id'])['product_id'].count().unstack().fillna(0)


In [11]:
unique_users.count()

Unnamed: 0                206209
order_id                  206209
user_id                   206209
eval_set                  206209
order_number              206209
order_dow                 206209
order_hour_of_day         206209
days_since_prior_order         0
product_id                206209
add_to_cart_order         206209
reordered                 206209
product_name              206209
aisle_id                  206209
department_id             206209
department                206209
aisle                     206209
dtype: int64

In [9]:
data_limited[(data_limited.user_id == 1) & (data_limited.product_id == 196)]

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,beverages,soft drinks
5,6,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,beverages,soft drinks
11,12,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,beverages,soft drinks
16,17,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,beverages,soft drinks
21,22,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,beverages,soft drinks
29,30,3367565,1,prior,6,2,7,19.0,196,1,1,Soda,77,7,beverages,soft drinks
33,34,550135,1,prior,7,1,9,20.0,196,1,1,Soda,77,7,beverages,soft drinks
39,40,3108588,1,prior,8,1,14,14.0,196,2,1,Soda,77,7,beverages,soft drinks
47,48,2295261,1,prior,9,1,16,0.0,196,4,1,Soda,77,7,beverages,soft drinks
50,51,2550362,1,prior,10,4,8,30.0,196,1,1,Soda,77,7,beverages,soft drinks


In [13]:
# converting the matrix back to a table for further processing
v_table = matrix.stack().reset_index()
v_table.columns = ['user_id','product_id','amount']

# remove rows with amount = 0
v_table = v_table[(v_table.amount != 0)]